# Assignment-1: Transformations and representations

Team Name: Bhagwaan Bharose

Roll number: 2019111019, 2019111026

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `Set Up` for detailed step-by-step instructions about the installation setup.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. 
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 11/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.


# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

# Set Up

We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. All assignments will be python based, hence familiarising yourself with Python is essential.


## Setting up Anaconda environment (Recommended)

1. Install Anaconda or Miniconda from [here](https://docs.conda.io/projects/conda/en/latest/user-guide/install/linux.html) depending on your requirements.
2. Now simply run `conda env create -f environment.yml` in the current folder to create an environment `mr_assignment1` (`environment.yml` can be found in `misc/`).
3. Activate it using `conda activate mr_assignment1`.

## Setting up Virtual environment using venv

You can also set up a virtual environment using venv

1. Run `sudo apt-get install python3-venv` from command line.
2. `python3 -m venv ~/virtual_env/mr_assignment1`. (you can set the environment path to anything)
3. `source ~/virtual_env/mr_assignment1/bin/activate`
4. `pip3 install -r requirements.txt` from the current folder (`requirements.txt` can be found in `misc/`).

In [1]:
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# 1. Getting started with Open3D

Open3D is an open-source library that deals with 3D data, such as point clouds, mesh. We'll be using Open3D frequently as we work with point clouds. Let's start with something simple:

<img src="misc/bunny.jpg" alt="drawing" width="200"/>

1. Read the Stanford Bunny file (in `data/`) given to you and visualise it using Open3D.
2. Convert the mesh to a point cloud and change the colour of points.
3. Set a predefined viewing angle (using Open3D) for visualization and display the axes while plotting.
4. Scale, Transform, and Rotate the rabbit (visualise after each step).
5. Save the point cloud as bunny.pcd.

In [ ]:
import copy

# Part 1 - Read the Stanford Bunny file (in data/) given to you and visualise it using Open3D.
mesh = o3d.io.read_triangle_mesh("./data/bunny.ply")
o3d.visualization.draw_geometries([mesh])
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])

# Part 2 - Convert the mesh to a point cloud and change the colour of points.
pcd = o3d.geometry.PointCloud()
pcd.points = mesh.vertices
pcd.colors = mesh.vertex_colors
pcd.normals = mesh.vertex_normals
o3d.visualization.draw_geometries([pcd])
pcd.paint_uniform_color([1, 0, 0])
o3d.visualization.draw_geometries([pcd])

# Part 3 - Set a predefined viewing angle (using Open3D) for visualization and display the axes while plotting.
FOR = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.025, origin=[0,0,0])
o3d.visualization.draw_geometries([pcd, FOR], 
                                  zoom=1, 
                                  front=[-0.01, 0.01, 0], 
                                  lookat=[-0.01, 0.01, 0],
                                  up=[0,1,0]
                                 )

# Part 4 - Scale, Transform, and Rotate the rabbit (visualise after each step).

#Scale
pcd.scale(1.5, center=pcd.get_center())
o3d.visualization.draw_geometries([pcd, FOR], 
                                  zoom=1, 
                                  front=[-0.01, 0.01, 0], 
                                  lookat=[-0.01, 0.01, 0],
                                  up=[0,1,0]
                                 )
# Transform
T = np.eye(4)
T[:3, :3] = pcd.get_rotation_matrix_from_xyz((0, np.pi / 3, np.pi / 2))
T[0, 3] = 0.1
T[1, 3] = 0.1
pcd_t = copy.deepcopy(pcd).transform(T)
pcd = copy.deepcopy(pcd_t)
o3d.visualization.draw_geometries([pcd, FOR], 
                                  zoom=1, 
                                  front=[-0.01, 0.01, 0], 
                                  lookat=[-0.01, 0.01, 0],
                                  up=[0,1,0]
                                 )
# Rotate
R = pcd.get_rotation_matrix_from_xyz((np.pi/2, 0, 0))
pcd.rotate(R, center=(0, 0, 0))
o3d.visualization.draw_geometries([pcd, FOR])


# Part 5 - Save the point cloud as bunny.pcd.
o3d.io.write_point_cloud("bunny.pcd", pcd)

# 2. Transformations and representations

## a) Euler angles
1. Write a function that returns a rotation matrix given the angles $\alpha$, $\beta$, and $\gamma$ in radians (X-Y-Z)

2. Solve for angles using ```fsolve from scipy``` for three initializations of your choice and compare.
$$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] 
$$

$$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] 
$$

3. What is a Gimbal lock? 

4. Show an example where a Gimbal lock occurs and visualize the Gimbal lock on the given bunny point cloud. You have to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*


In [65]:
# check rotation matrix
# https://answers.opencv.org/question/161369/retrieve-yaw-pitch-roll-from-rvec/

import numpy as np
import math

def rotation_matrix(theta):
    R_x = np.array([[1, 0, 0],
                    [0, math.cos(theta[0]), -math.sin(theta[0])],
                    [0, math.sin(theta[0]), math.cos(theta[0])]
                   ])
    R_y = np.array([[math.cos(theta[1]), 0, math.sin(theta[1])],
                    [0, 1, 0],
                    [-math.sin(theta[1]), 0, math.cos(theta[1])]
                    ])
    R_z = np.array([[math.cos(theta[2]), -math.sin(theta[2]), 0],
                    [math.sin(theta[2]), math.cos(theta[2]), 0],
                    [0, 0, 1]
                    ])
    gg = np.dot(np.dot(R_x,R_y), R_z)
    return gg.round(9)

alpha=np.pi/2
beta=np.pi/4
gamma=np.pi/3
thetas=np.array([alpha, beta, gamma])
Sa=np.sin(thetas[0])
Ca=np.cos(thetas[0])
Sb=np.sin(thetas[1])
Cb=np.cos(thetas[1])
Sc=np.sin(thetas[2])
Cc=np.cos(thetas[2])

R=rotation_matrix([alpha, beta, gamma])
print(R)

print("- - - - - ")
print("- - - - - ")
print("- - - - - ")

r = R
r[0][0] = Cb*Cc
r[0][1] = -Cb*Sc
r[0][2] = Sb
r[1][0] = Sa*Sb*Cc+Ca*Sc
r[1][1] = -Sa*Sb*Sc+Ca*Cc
r[1][2] = -Sa*Cb
r[2][0] = -Ca*Sb*Cc + Sa*Sc
r[2][1] = Ca*Sb*Sc+Sa*Cc
r[2][2] = Ca*Cb

print(r.round(9))

print("-------")

print(np.arctan2(R[1][0]/np.cos(beta),R[0][0]/np.cos(beta)))
print(np.arctan2(-R[2][0],(np.sqrt(R[0][0]**2+R[1][0]**2))))
print(np.arctan2(R[2][1]/np.cos(beta),R[2][2]/np.cos(beta)))

print([thetas])

[[ 0.35355339 -0.61237244  0.70710678]
 [ 0.35355339 -0.61237244 -0.70710678]
 [ 0.8660254   0.5         0.        ]]
- - - - - 
- - - - - 
- - - - - 
[[ 0.35355339 -0.61237244  0.70710678]
 [ 0.35355339 -0.61237244 -0.70710678]
 [ 0.8660254   0.5         0.        ]]
-------
0.7853981633974483
-1.0471975511965976
1.5707963267948966
[array([1.57079633, 0.78539816, 1.04719755])]


In [62]:
import numpy as np
import math
from scipy.optimize import fsolve

def rotation_equations(thetas):
    
    # check if R[0][2] is 1 or -1
    
    Sa=np.sin(thetas[0])
    Ca=np.cos(thetas[0])
    Sb=np.sin(thetas[1])
    Cb=np.cos(thetas[1])
    Sc=np.sin(thetas[2])
    Cc=np.cos(thetas[2])
    
    alpha=thetas[0]
    beta=thetas[1]
    gamma=thetas[2]
    
    return [
        alpha - np.arctan2(R[1][0]/np.cos(beta),R[0][0]/np.cos(beta)),
        beta - np.arctan2(R[0][2],(np.sqrt(R[0][0]**2+R[0][1]**2))),
        gamma - np.arctan2(R[2][1]/np.cos(beta),R[2][2]/np.cos(beta))
    ]
        
#     return [Cb*Cc-Cb*Sc+Sb-R[0][0]-R[0][1]-R[0][2],
#             Sa*Sb*Cc+Ca*Sc-Sa*Sb*Sc+Ca*Cc-Sa*Cb-R[1][0]-R[1][1]-R[1][2],
#             -Ca*Sb*Cc + Sa*Sc+Ca*Sb*Sc+Sa*Cc+Ca*Cb-R[2][0]-R[2][1]-R[2][2]
#            ]

#     return [Sa*Sb*Cc+Ca*Sc-R10,
#             -Sa*Sb*Sc+Ca*Cc-R11,
#             -Sa*Cb-R12]

x = fsolve(rotation_equations, [-1,-1,-1])
print((x%np.pi).round(3))
print(thetas%np.pi)

[3.142 0.    1.571]
[1.57079633 0.         0.        ]


## b) Quaternions

1. What makes Quaternions popular in graphics? 
2. Convert a rotation matrix to quaternion and vice versa. Do not use inbuilt libraries for this question.
3. Perform matrix multiplication of two $\mathcal{R}_{3 \times 3}$ rotation matrices and perform the same transformation in the quaternion space. Verify if the final transformation obtained in both the cases are the same.
4. Try to interpolate any 3D model (cube / bunny / not sphere obviously!!) between two rotation matrices and visualize!

The above questions require you to **code your own functions** and **only verify** using inbuilt functions.

## c) Exponential maps (Bonus)

1. What is the idea behind exponential map representation of rotation matrices?
2. Perform matrix exponentiation and obtain the rotation matrix to rotate a vector $P$ around $\omega$ for $\theta$ seconds.
$$
\omega = \begin{bmatrix}2 \\ 1 \\ 15 \end{bmatrix}
$$

$$
\theta = 4.1364
$$

3. Compute the logarithmic map (SO(3) to so(3)) of the rotation matrix to obtain the rotation vector and the angle of rotation
$$
\begin{bmatrix}
0.1 &  -0.9487 & 0.3 \\
0.9487 & 0.  & -0.3162 \\
0.3   &  0.3162  & 0.9 
\end{bmatrix}
$$
You can use inbuilt libraries **only to verify** your results.

# 3. Data representations

## a) Octomaps

1. Why is an Octomap memory efficient?
2. When do we update an Octomap and why?
3. When would you likely use an octomap instead of a point cloud?
 

## b) Signed Distance Functions

1. How do we determine object surfaces using SDF?
2. How do we aggregate views from multiple cameras? (just a general overview is fine)
3. Which preserves details better? Voxels or SDF? Why?
4. What’s an advantage of SDF over a point cloud?


# References and Resources

1. Gimbal locks and quaternions: https://youtu.be/YF5ZUlKxSgE
2. Exponential map: 
    1. 3 Blue 1 Brown: https://youtu.be/O85OWBJ2ayo
    2. Northwestern Robotics: https://youtu.be/v_KBHaG0mas
3. Bunny ply is taken from: http://graphics.im.ntu.edu.tw/~robin/courses/cg03/model/